CREW AI AGENT : `Financial Reporter`
---

An advanced research assistant by plugging LlamaIndex-powered tools into a CrewAI-powered multi-agent setup.

LlamaIndex is a framework enabling developers to easily build LLM-powered applications over their data; it contains production modules for indexing, retrieval, and prompt/agent orchestration. A core use case is building a generalized QA interface enabling knowledge synthesis over complex questions.

Plugging a LlamaIndex RAG pipeline as a tool into a CrewAI agent setup enables even more sophisticated/advanced research flows

FINANCIAL ANALYST `LANGCHAIN & CREW AI`
---

In [1]:
# Install
!pip install -q 'crewai[tools]' --progress-bar off
!pip install -q langchain-groq --progress-bar off
!pip install -q duckduckgo-search --progress-bar off
!pip install -q yfinance --progress-bar off

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [145]:
#import os
#from google.colab import userdata
#os.environ["GROQ_API"] = userdata.get["GROQ_API"]

In [9]:
from google.colab import userdata
import pandas as pd
import requests as re
from datetime import datetime
import yfinance as yf
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from langchain.agents import load_tools
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchResults #DuckDuckGoSearchRunTool DuckDuckGoSearchResults DuckDuckGoSearchRun
from langchain_groq import ChatGroq

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key=userdata.get('GROQ_API')
)

In [11]:
def format_response(response: str) -> str:
    entries = response.split(r"(?<=]), (?=\[)", response)
    return [entry.strip("[]") for entry in entries]

In [12]:
# Search tool
search_tool = DuckDuckGoSearchResults(backend="news", num_results=5 , verbose=True)
response = search_tool.run("PLTR")
print(response)

[snippet: Since Palantir Technologies Inc (NYSE:PLTR) ranks 12th on the list, it deserves a deeper look. Wedbush Securities analyst Dan Ives has said in a fresh note that major tech giants are now running in a "race" to capture the $4 trillion AI market ..., title: Is Palantir Technologies Inc (NYSE:PLTR) the Best AI Stock in the $4 Trillion 'AI Race', link: https://finance.yahoo.com/news/palantir-technologies-inc-nyse-pltr-160147482.html, date: 2024-06-24T12:01:00+00:00, source: Yahoo Finance], [snippet: C3.ai is, in my opinion, a lousy AI stock overall in one to generally avoid. Why not instead choose Palantir (NYSE:PLTR)? It's a profitable option in the enterprise AI space. C3.ai is not even close in the race of AI stocks to buy and sell. On the date ..., title: Spotlight on 3 Popular AI Stocks: 2 Studs and One Dud, link: https://markets.businessinsider.com/news/stocks/spotlight-on-3-popular-ai-stocks-2-studs-and-one-dud-1033503210, date: 2024-06-25T05:15:00+00:00, source: Business 

In [25]:
# Tool Financial data tool
@tool("financial data tool")
def financial_data_tool(ticker):
    """
    Get all the financial data metrics for a given stock ticker name
    """
    # Fetch data for the company
    stock = yf.Ticker(ticker)
    # Initialize the document with the company's ticker
    document = f"Financial Data informations listed by the company with ticker {ticker} :\n ---- \n"
    # Get the info and reco
    document += "\n---\n ## Info and Financial metrics : \n" + str(stock.info)
    document += "\n---\n ## Recommendations : \n" + str(stock.recommendations)
    # Add news to the document
    news = stock.news
    out = "\n---\n ## News for the stock : \n"
    for i in news:
        out += f"# {i['title']} \n Publisher : {i['publisher']} \n Related ticker : {i['relatedTickers']} \n "
    document += out
    # balancesheet
    balance_sheet = pd.DataFrame(stock.balance_sheet)
    document += "\n---\n ## Balance Sheet : \n" + balance_sheet.to_string()
    # cash flow
    cash_flow = pd.DataFrame(stock.cashflow)
    document += "\n---\n ## Cash Flow : \n" + cash_flow.to_string()
    # income statement
    income_statement = pd.DataFrame(stock.income_stmt)
    document += "\n---\n ## Income Statement : \n" + income_statement.to_string()

    return document


In [23]:
# Tool News tool
@tool("search news tool")
def search_news_tool(ticker):
  """
  Get the last news about a given stock ticker name
  """
  search_tool = DuckDuckGoSearchResults(backend="news", num_results=5 , verbose=True)
  return search_tool.run(ticker)


In [28]:
# Tool : Human Tool
humain_tool = load_tools(["human"])

In [29]:
# Define your agents with roles and goals
assistant = Agent(
    role="Customer assistant",
    goal="Identify which {ticker} the customer is interested in",
    backstory="""You a highly Customer assistant helping customer to pick a stock {ticker},
    based on customer interest by domain sector.
    """,
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    tools=humain_tool,
    llm = llm,
)

# financial analyst
analyst = Agent(
    role="Financial Analyst",
    goal="Uncover main financial situation and insights about company {ticker}",
    backstory="""You work at an asset management firm analyst.
  Your goal is to understand the stock {ticker} based of the financial data and the current market condition.""",
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    tools=[financial_data_tool],
    llm = llm,
)

researcher = Agent(
    role="News Analyst",
    goal="Uncover news about the company {ticker}",
    backstory="""You work at an news office for asset management firm.
  Your goal is to get and categorize news of the stock {ticker} and identify trends.""",
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    tools=[search_news_tool],
    llm = llm,
)
reporter = Agent(
    role="Financial Reporter",
    goal="Craft compelling and detailed content on financial report for the stock {ticker}",
    backstory="""You are a renowned Content Strategist for {ticker}, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    memory=True,
    max_iter=5,
    allow_delegation=False,
    llm = llm,
)


In [30]:
# Create tasks for your agents
assistant_task_0 = Task(
    description="""
    Once you have the ticker : stop asking and proceed to the next step.
    Start by asking if the user is interested in a specific stock ticker.
    If not help it by asking a specific sector/industry he is intersted in. and provide choice between 5 stocks in this sector.
    Include a feature to provide a brief overview of the stock’s performance over the past month to give the user a starting point for their decision-making.
    """,
    expected_output="{ticker} symbol (eg. : AAPL, IBM, BAC)",
    agent=assistant,
)

assistant_task = Task(
    description="""
    Begin by confirming the user’s interest in a specific stock ticker.
    If the user is unsure, assist by inquiring about their preferred sector or industry (e.g., AI, cloud, energy).
    Then, offer a selection of five stocks within that sector for consideration.
    """,
    expected_output="{ticker} symbol (eg. : AAPL, IBM, BAC)",
    agent=assistant,
)

analyst_task = Task(
    description=f"""
    Use the financial data tool to get the main financial situation and insights about company ticker.
    as today as {datetime.now()}
    In a Section : Start by providing a brief overview of the market where stock is operating, description of the stock company, the main products, competitors.
    develop the financial report in sections : Perform an in-depth analysis and present the main data of the Stock with sections (analysis section and data table below): analysis of the price, valuation , performance, debt situation, short situation and analysis recommandations, then highlights the headwinds and tailwinds that stocks faces.
    Incorporate a comparison with industry benchmarks to give context to the stock’s performance.
    Provide a conclusion to help the user make a decision.
    Your report should be lowlevel detailed and informative, with paragraphs and data tables, catering to a professional audience.
    Make it sound professional, provide full analysis.
    """,
    expected_output="Full financial report for {ticker} in markdown format of at least 8 paragraphs",
    agent=assistant,
    context=[assistant_task],
    output_file='analyst_report.md',
)

research_task = Task(
    description=f"""
    Use the search news tool to get the last news about company ticker, as today as {datetime.now()}
    Perform an in-depth analysis of these news, focusing on identifying potential risks and headwinds, as well as forecasting future trends.
    Categorize and display the news into Positive/Bullish, Negative/Bearish, Neutral.
    Provide a thorough news analysis report for the ticker, presented in bullet points, highlighting risks and tailwinds, insights, and future trends.
    Give a conclusion at the end of the report highlighting the main news and trends.
    """,
    expected_output="Full News analysis report for {ticker} with news by catagory and risks, tailwinds, insights, futur trends. in bullet points",
    agent=researcher,
    context=[assistant_task],
    output_file='news_report.md',
)

reporter_task = Task(
    description=f"""
    Use both reports provided by the Financial Analyst and the News Analyst to create the final financial report.
    The current date is {datetime.now()}
    Provide a summary at the beginning of the report with key takeaways for quick reference.
    Present first the the Financial Analyst report and the News Analyst report in markdown format.
    Add a section on market sentiment analysis using news trends to provide a more comprehensive view.
    Conduct a Highly detailed analysis of ticker's financial data.
    look for the risks and tailwinds and identify futur trends.
    In a Data Annexe Section : Provide a brief overview of the stock’s main finance data in table format to give the user more precices inputs.
    Conculsion at the end of the report highlighting the main news and trends.
    """,
    expected_output="Full Financial report for {ticker} in rich markdown format, Your report should be lowlevel detailed and informative, catering to a professional audience. Make it sound professional, provide full analysis, introduction and conclusion",
    agent=researcher,
    context=[analyst_task, research_task],
    output_file='financial_report.md',
)

In [36]:
# the Ai Crew
crew = Crew(
    agents=[assistant, analyst, researcher, reporter],
    tasks=[assistant_task, analyst_task, research_task, reporter_task],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    #llm=llm_groq,
    manager_llm=llm,
    max_iter=24,
    #memory=True,
    #cache=True,
    )


In [37]:
# Run the Crew
result = crew.kickoff()

 [2024-06-25 12:20:59][DEBUG]: == Working Agent: Customer assistant
 [2024-06-25 12:20:59][INFO]: == Starting Task: 
    Begin by confirming the user’s interest in a specific stock ticker. 
    If the user is unsure, assist by inquiring about their preferred sector or industry (e.g., AI, cloud, energy). 
    Then, offer a selection of five stocks within that sector for consideration.
    


> Entering new CrewAgentExecutor chain...
Thought: I need to confirm the user's interest in a specific stock ticker or sector.

Action: human
Action Input: {"question": "Are you interested in a specific stock ticker or sector?"}

Are you interested in a specific stock ticker or sector?
yes, report OKTA please
 

yes, report OKTA please

Thought: The user is interested in a specific stock ticker, OKTA, but I'd like to confirm if they have a preferred sector or industry.

Action: human
Action Input: {"question": "Is OKTA aligned with a specific sector or industry that you're interested in, such as cyb

DATA : YAHOO FINANCE TOOL
---

In [39]:
import yfinance as yf
import pandas as pd

def write_financial_data(ticker):
    # Fetch data for the company
    stock = yf.Ticker(ticker)
    # Initialize the document with the company's ticker
    document = f"Financial Data informations listed by the company with ticker {ticker} :\n ---- \n"
    # Get the info and reco
    document += "\n---\n ## Info and Financial metrics : \n" + str(stock.info)
    document += "\n---\n ## Recommendations : \n" + str(stock.recommendations)
    # Add news to the document
    news = stock.news
    out = "\n---\n ## News for the stock : \n"
    for i in news:
        out += f"# {i['title']} \n Publisher : {i['publisher']} \n Related ticker : {i['relatedTickers']} \n "
    document += out
    # balancesheet
    balance_sheet = pd.DataFrame(stock.balance_sheet)
    document += "\n---\n ## Balance Sheet : \n" + balance_sheet.to_string()
    # cash flow
    cash_flow = pd.DataFrame(stock.cashflow)
    document += "\n---\n ## Cash Flow : \n" + cash_flow.to_string()
    # income statement
    income_statement = pd.DataFrame(stock.income_stmt)
    document += "\n---\n ## Income Statement : \n" + income_statement.to_string()

    # write to file in ./ dir
    with open(f'financial_data_{ticker}.txt', 'w') as file:
        file.write(document)
    print(f'Document loaded and saved : financial_data_{ticker}.txt')

    return document

----

DATA : TAVILY TOOL
---

In [ ]:
# Getting Data from YF and index
GROQ_API = userdata.get('TAVILY_API')

tavily_tool = TavilyToolSpec(api_key=TAVILY_API)
tavily_tool_list = tavily_tool.to_tool_list()
#chat_engine = ReActAgent.from_tools(tools=tavily_tool_list)

PROMPT HELP
---

# Certainly! Here are some enhancements for the tasks you’ve outlined:

Task 1: Agent Task Enhancement

Add a step to verify the user’s interest in the stock ticker by asking for confirmation before proceeding.
Include a feature to provide a brief overview of the stock’s performance over the past month to give the user a starting point for their decision-making.
Task 2: Analyst Task Enhancement

Incorporate a comparison with industry benchmarks to give context to the stock’s performance.
Add a section on market sentiment analysis using social media and news trends to provide a more comprehensive view.
Task 3: Research Task Enhancement

Expand the search to include international news sources for a global perspective.
Implement sentiment analysis on the news articles to gauge public perception of the company.
Task 4: Reporter Task Enhancement

Integrate visual aids such as charts and graphs to illustrate financial data and trends.
Provide a summary at the beginning of the report with key takeaways for quick reference.
These enhancements aim to provide a more robust and informative experience for users interested in stock analysis. If you need further assistance or have specific requirements, feel free to let me know! 😊



#Absolutely, I'd be happy to help you refine the descriptions and expected outputs for your tasks. Here's how you can enhance them:

**Task 1: Agent Task**
- Description: Begin by confirming the user's interest in a specific stock ticker. If the user is unsure, assist by inquiring about their preferred sector or industry (e.g., AI, cloud, energy). Then, offer a selection of five stocks within that sector for consideration.
- Expected Output: The user should provide a stock ticker symbol (e.g., AAPL, IBM, BAC).

**Task 2: Analyst Task**
- Description: Utilize the financial data tool to gather comprehensive insights into the company's financial health and market position. As of today's date, craft a detailed financial report that includes an analysis of the stock's price, valuation, and performance. Additionally, identify and discuss any challenges or headwinds that the company may be facing.
- Expected Output: A professional-grade financial report for the specified ticker in markdown format, comprising at least eight paragraphs.

**Task 3: Research Task**
- Description: Employ the search news tool to retrieve the latest news articles related to the company ticker. Perform an in-depth analysis of these articles, focusing on identifying potential risks and headwinds, as well as forecasting future trends.
- Expected Output: A thorough news analysis report for the ticker, presented in bullet points, highlighting risks, tailwinds, insights, and future trends.

**Task 4: Reporter Task**
- Description: Integrate the findings from both the financial analyst and news analyst to compile a comprehensive financial report. This report should include a detailed analysis of the ticker's recent news, risks, headwinds, and future trends. Ensure that the report is presented in a professional manner with an introduction and conclusion.
- Expected Output: A rich markdown format financial report for the ticker that is detailed and informative, suitable for a professional audience. The report should include financial data analysis, risks, tailwinds, insights, and future trends.

These enhancements aim to provide clarity and structure to your tasks while ensuring that they are informative and engaging for your audience. If you need further assistance or have additional questions, feel free to ask! 😊